In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
TRAIN_PATH = 'digital_reputation_challenge_train/train/'
TEST_PATH = 'digital_reputation_challenge_test/test/'

Загрузим данные

In [3]:
d_train, d_test = {}, {}
all_urls = []
all_urls_num = 0

In [4]:
with open(TRAIN_PATH + 'Y.csv', 'r') as f:
    for line in f:
        if 'id' in line:
            continue
        lst = line[:-1].split(',')
        d_train[lst[0]] = [np.array([float(s) for s in lst[1:]])]

In [5]:
with open(TRAIN_PATH + 'X2.csv', 'r') as g:
    for line in g:
        if 'id' in line:
            continue
        idx, site = line[:-1].split(',')
        d_train[idx].append(site)
        all_urls.append(site)
        all_urls_num += 1

In [6]:
with open(TEST_PATH + 'X2.csv', 'r') as f:
    for line in f:
        if 'id' in line:
            continue
        idx, site = line[:-1].split(',')
        if idx not in d_test.keys():
            d_test[idx] = [site]
        else:
            d_test[idx].append(site)

In [7]:
import collections
all_urls = collections.Counter(all_urls)

In [8]:
d = {'id': [], '1': [], '2': [], '3': [], '4': [], '5': []}
for test_id in d_test.keys():
    num = 0.
    res = np.array([0., 0., 0., 0., 0.])
    for train_id in d_train.keys():
        common = set(d_train[train_id][1:]) & set(d_test[test_id])
        m = 0
        for url in common: 
            m += ((math.log(all_urls_num) - math.log(all_urls[url])) - 6)
        res += d_train[train_id][0] * m
        num += m
    d['id'].append(test_id)
    if num < 1:
        for i in '12345':
            d[i].append(0.3)
    else:
        res /= num
        for i in range(1, 6):
            d[str(i)].append(res[i - 1])

In [9]:
first_solution_result = pd.DataFrame(data=d)

In [10]:
first_solution_result

,id,1,2,3,4,5
0,0,0.244429,0.279312,0.296971,0.328332,0.362890
1,1,0.297182,0.282895,0.330458,0.290513,0.414352
2,2,0.319323,0.299394,0.297163,0.259389,0.371263
3,4,0.244553,0.244649,0.381502,0.274347,0.536876
4,7,0.288297,0.313971,0.389969,0.351190,0.339812
...,...,...,...,...,...,...
4053,8046,0.332901,0.344083,0.395553,0.377831,0.358500
4054,8047,0.290145,0.368577,0.362799,0.395955,0.341957
4055,8050,0.225583,0.188553,0.406626,0.281559,0.477991
4056,8054,0.181324,0.112692,0.403090,0.277541,0.509542


In [11]:
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import (KFold, StratifiedKFold, cross_val_score,
                                     cross_validate, train_test_split)
from xgboost import XGBClassifier

In [12]:
X1 = pd.read_csv(TRAIN_PATH + 'X1.csv')
X1_test = pd.read_csv(TEST_PATH + 'X1.csv')
Y = pd.read_csv(TRAIN_PATH + 'Y.csv')

In [13]:
X1['2'] = [int(val) for val in X1['2']]
X1['3'] = [int(val) for val in X1['3']]
X1_test['2'] = [int(val) for val in X1_test['2']]
X1_test['3'] = [int(val) for val in X1_test['3']]

In [14]:
d_mx = {'min_child_samples': 100, 'learning_rate': 0.010, 'max_depth': 3, 'max_delta_step': 2, 'num_leaves': 12, 'min_child_weight': 2, 'n_estimators': 195}
d_mx_2 = {'min_child_samples': 100, 'learning_rate': 0.019, 'max_depth': 1, 'max_delta_step': 2, 'num_leaves': 12, 'min_child_weight': 2, 'n_estimators': 200}
d_mx_3 = {'min_child_samples': 100, 'learning_rate': 0.012, 'max_depth': 2, 'max_delta_step': 2, 'num_leaves': 12, 'min_child_weight': 2, 'n_estimators': 195}
d_mx_4 = {'learning_rate': 0.018, 'max_delta_step': 2, 'max_depth': 1, 'min_child_samples': 150, 'min_child_weight': 7, 'n_estimators': 315, 'num_leaves': 20}

In [15]:
cls = []
cls.append(XGBClassifier(n_jobs=8, random_state=0, objective='rank:pairwise', **d_mx))
cls.append(XGBClassifier(n_jobs=8, random_state=0, **d_mx_2))
cls.append(XGBClassifier(n_jobs=8, random_state=0, objective='rank:pairwise', **d_mx_3))
cls.append(XGBClassifier(n_jobs=8, random_state=0, **d_mx_4))

In [16]:
probas = []
for i in range(4):
    cls[i].fit(X1.values, Y[str(i+1)].values)
    proba = cls[i].predict_proba(X1_test.values)[:, 1]
    probas += [proba]

In [17]:
second_solution_result = pd.DataFrame(probas).T

In [18]:
second_solution_result

,0,1,2,3
0,0.584335,0.346478,0.141240,0.275740
1,0.664818,0.258636,0.244932,0.256850
2,0.770092,0.437305,0.089138,0.275740
3,0.463854,0.312374,0.095047,0.263528
4,0.701586,0.258636,0.785090,0.363445
...,...,...,...,...
4053,0.458925,0.276307,0.824031,0.370102
4054,0.252209,0.344807,0.512855,0.398014
4055,-0.012030,0.248609,0.495471,0.250037
4056,0.135481,0.344807,0.647099,0.377602


In [19]:
coef_list = [0.3, 0.7, 0.3, 0.7]

for i in range(4):
    
    first_lst = first_solution_result[str(i + 1)]
    first_np = np.array(first_lst)
    first_lst = [(first_np > val).sum() for val in first_lst]

    second_lst = second_solution_result[i]
    second_np = np.array(second_lst)
    second_lst = [(second_np > val).sum() for val in second_lst]
    
    final_ranks = (1 - (np.array(first_lst) * coef_list[i] + np.array(second_lst) * (1 - coef_list[i])) / 4060).tolist()
    first_solution_result[str(i + 1)] = final_ranks

In [20]:
first_solution_result.to_csv('final.csv', index=False)